In [3]:
import pandas as pd

In [13]:
from nba_api.stats.static import teams
# get_teams returns a list of 30 dictionaries, each an NBA team.
nba_teams = pd.DataFrame(teams.get_teams())
#print('Number of teams fetched: {}'.format(len(nba_teams)))
nba_teams

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [16]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
# Select the dictionary for the Mavs, which contains their team ID
Mavs = [team for team in nba_teams if team['abbreviation'] == 'DAL'][0]
Mavs_id = Mavs['id']

In [17]:
from nba_api.stats.endpoints import leaguegamefinder

# Query for games where the Mavs were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=Mavs_id)
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22021,1610612742,DAL,Dallas Mavericks,0022101079,2022-03-21,DAL vs. MIN,W,241,110,...,0.762,7,40,47,27,5,3,11,21,2.0
1,22021,1610612742,DAL,Dallas Mavericks,0022101059,2022-03-19,DAL @ CHA,L,241,108,...,0.800,12,32,44,18,4,4,13,26,-21.0
2,22021,1610612742,DAL,Dallas Mavericks,0022101046,2022-03-18,DAL @ PHI,L,241,101,...,0.714,11,29,40,21,4,3,12,17,-10.0
3,22021,1610612742,DAL,Dallas Mavericks,0022101036,2022-03-16,DAL @ BKN,W,239,113,...,1.000,9,28,37,21,5,2,10,17,2.0
4,22021,1610612742,DAL,Dallas Mavericks,0022101014,2022-03-13,DAL @ BOS,W,241,95,...,0.737,4,38,42,20,7,5,17,16,3.0


In [19]:
gamefinder = leaguegamefinder.LeagueGameFinder()
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.tail()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
29995,22015,1612709919,WES,Westchester Knicks,2021500202,2016-01-19,WES @ DEL,W,241,127,...,0.681,11,42,53,20,13,6,18,24,15.0
29996,22015,1610612748,MIA,Miami Heat,0021500625,2016-01-19,MIA vs. MIL,L,239,79,...,0.769,12,26,38,13,6,2,14,14,-12.0
29997,22015,1612709904,SXF,Sioux Falls Skyforce,2021500203,2016-01-19,SXF vs. IWA,W,255,119,...,0.782,12,29,41,18,11,5,13,30,7.0
29998,22015,1610612756,PHX,Phoenix Suns,0021500628,2016-01-19,PHX vs. IND,L,241,94,...,0.931,13,40,53,15,4,2,22,23,-3.0
29999,22015,1610612749,MIL,Milwaukee Bucks,0021500625,2016-01-19,MIL @ MIA,W,242,91,...,0.875,7,32,39,19,6,4,17,19,12.0
